In [46]:

import os
from pyarrow import feather
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import torch
import pandas as pd

In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
def write_to_feather(df, name):
    root_dir = os.path.dirname(os.path.abspath('twitter_sentimentAnalysis.ipynb'))
    path = os.path.join(root_dir, 'twitter_res/' + name)
    feather.write_feather(df, path)

In [40]:
df_twitter_old = pd.read_feather("twitter_res/twitter_old.ftr");
df_twitter_new = pd.read_feather("twitter_res/twitter_new.ftr");

In [41]:
print(df_twitter_old['tweet'][0]);

 climate change interesting hustle global warming planet stopped warming 15 yes suv boom 


In [48]:
counter = 0
allScores = [[] for i in range(len(df_twitter_old['tweet']))]

for tweet in df_twitter_old['tweet']:
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores) #scores = [negative, neutral, positive] -> Are always in this order
    #print(f'{counter}: {scores}')
    allScores[counter] = scores
    counter += 1

df_twitter_old['sentimentScore'] = allScores
df_twitter_old

,tweet,tweet_ts,hashtags,sentimentScore
0,climate change interesting hustle global warm...,31-10-2016,[],"[0.013433724, 0.56145763, 0.42510867]"
1,"watch #beforetheflood right here, travels wo...",31-10-2016,[#beforetheflood],"[0.019552002, 0.795175, 0.18527298]"
2,fabulous! leonardo #dicaprio's film #climate c...,31-10-2016,"[#dicaprio, #climate]","[0.0022631218, 0.01210344, 0.98563343]"
3,watched amazing documentary leonardodicaprio ...,31-10-2016,[],"[0.0034224496, 0.031758707, 0.9648187]"
4,"pranita biswasi, lutheran odisha, gives testi...",31-10-2016,[],"[0.021988245, 0.9171695, 0.06084218]"
...,...,...,...,...
43934,#awareness walls aren$q$t answer people fleein...,26-10-2016,[#awareness],[]
43935,americans scared clowns climate change.,26-10-2016,[],[]
43939,respective parties prevent climate change glob...,26-10-2016,[#zpndebate],[]
43941,still can$q$t believe gif taehyung saved huma...,26-10-2016,[],[]


In [49]:
#save the data
write_to_feather(df_twitter_old, 'twitter_old_analysis.ftr')

In [51]:
#df_twitter_test = pd.read_feather("twitter_res/twitter_old_analysis.ftr");
#df_twitter_test

In [37]:
df_twitter_old['sentimentScore'][0][0]

0.013433724

In [52]:
counter = 0
allScores = [0 for i in range(len(df_twitter_new['tweet']))]

for tweet in df_twitter_new['tweet']:
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores) #scores = [negative, neutral, positive]
    #print(f'{counter}: {scores}')
    allScores[counter] = scores
    counter += 1

df_twitter_new['sentimentScore'] = allScores;
df_twitter_new

,tweet,tweet_ts,hashtags,sentimentScore
0,lawmakers justify protecting old growth forest...,11-11-2022,"[#scicomm, #carbon, #climatechange]","[0.7790241, 0.21138299, 0.009592833]"
1,evolution plant drought strategies herbivore t...,28-04-2022,[#climatechange],"[0.050136503, 0.91367716, 0.036186364]"
2,#conservation #climatecrisis #climateaction #r...,14-03-2022,"[#conservation, #climatecrisis, #climateaction...","[0.067462675, 0.8687227, 0.063814655]"
3,"winner year's photography4humanity prize, ""a t...",04-01-2022,[#climateaction],"[0.056534603, 0.685811, 0.25765446]"
4,"blame shifting, peak passivity, lies #climatec...",10-03-2022,"[#climatecrisis, #scottydoesnothing]","[0.884136, 0.107250676, 0.008613364]"
...,...,...,...,...
2363325,world temperature anomalies 17.03.2023 76 /76 ...,17-03-2023,"[#arctic, #africa, #russia, #climatecrisis, #c...",0
2363326,louisiana 8 worst water-polluting refineries c...,31-01-2023,"[#climatechange, #oceans, #oceanpullution, #po...",0
2363328,"free webinar: electric vehicles 101, january 3...",26-01-2023,"[#evs, #electricvehicles, #vehicles, #cars, #p...",0
2363329,#worldozoneday #saveearth #gogreen #savesoil ...,16-09-2022,"[#worldozoneday, #saveearth, #gogreen, #saveso...",0


In [ ]:
#save the data
write_to_feather(df_twitter_new, 'twitter_old_analysis.ftr')